In [1]:
from newspaper import Article
import pandas as pd
import requests
import numpy as np
from datetime import datetime as dt
import time as tm
import re
from bs4 import BeautifulSoup
import multiprocessing as mp

In [2]:
chosen = pd.read_csv('final_chosen.csv')
folder= 'chosenSamp'

In [3]:
def overwrite(path):

    print('async hit')

    f = open(path, 'rb')
    article = Article('')
    article.download(f.read())
    article.parse()

    ret = []

    try:
        ret.append(article.title)
        ret.append(article.publish_date)
        ret.append(article.text)
        if len(article.authors) > 0: 
            ret.append(article.authors[0])
    except:
        print('couldnt parse: ', path)

    return ret

results = []
paths = []

def collect_result(result):
    global results
    results.append(result)

pool = mp.Pool(mp.cpu_count())

for index, row in chosen.iterrows():

    path = folder + '/' + ''.join(re.findall('\d+', row.date)) + '_' + ''.join(re.findall('(\d+|[a-zA-Z]+|-|\.)', row.url)) + '.html'
    paths.append(path)

pool.map_async(overwrite, paths, callback=collect_result)
pool.close()
pool.join()

In [ ]:
display(results)

'2019-10-08'

In [13]:

# this creates and sets all the new variables to None
chosen['cleaned'] = None
chosen['title'] = None
chosen['published'] = None
chosen['author'] = None

def parse(collection_id, data, folder):
    ctr=0

    # the base url for the archive
    ai_url = "https://wayback.archive-it.org"

    # this loop iterates through all the urls, searches for the archived webpage
    # then looks for any hyperlinks on the page that are associated with a date in the list
    for index, row in data.iterrows():

        ctr+=1
        # if ctr > 10: break

        # this is the url of an archived webpage's home 
        # (e.g. https://wayback.archive-it.org/12706/*/http://mypaperonline.com)
        path = folder + '/' + ''.join(re.findall('\d+', row.date)) + '_' + ''.join(re.findall('(\d+|[a-zA-Z]+|-|\.)', row.url)) + '.html'
            
        f = open(path, 'rb')
        article = Article('')
        article.download(f.read())
        article.parse()

        try:
            data.loc[index, 'title'] = article.title
            if len(article.authors) > 0: 
                data.loc[index, 'author'] = article.authors[0]
            data.loc[index, 'published'] = article.publish_date
            data.loc[index, 'text'] = article.text
            data.loc[index, 'cleaned'] = 1
        except:
            data.loc[index, 'cleaned'] = 0

    return data

In [19]:
start = tm.time()
new_parse = parse(12706, chosen, 'chosenSamp')
span = tm.time() - start
print('completed in ', span, ' seconds')

ArticleException: You must `download()` an article first!

In [ ]:
new_parse.to_csv('new_test_parsing.csv')

In [ ]:
# this creates and sets all the new variables to None
chosen['cleaned'] = None
chosen['title'] = None
chosen['published'] = None
chosen['author'] = None
chosen['date'] = pd.to_datetime(chosen['date'])

# this function constructs the url for an archived webpage according to https://wayback.archive-it.org formatting
def old_parse(collection_id, data):
    ctr=0

    # the base url for the archive
    ai_url = "https://wayback.archive-it.org"

    # this loop iterates through all the urls, searches for the archived webpage
    # then looks for any hyperlinks on the page that are associated with a date in the list
    for index, row in data.iterrows():

        ctr+=1
        # if ctr > 10: break

        # this is the url of an archived webpage's home 
        # (e.g. https://wayback.archive-it.org/12706/*/http://mypaperonline.com)
        home = ai_url + '/' + str(collection_id) + '/*/' + row.url

        # this block gets all the date hyperlinks
        page = requests.get(home)
        soup = BeautifulSoup(page.content, "lxml")
        alist = soup.find_all("a")
        alist = [a for a in alist if 'onclick' in a.attrs]
        dates = [dt.strptime(a.text, '%b %d, %Y') for a in alist]

        # this loops through all the date hyperlinks and if they're in
        # the date_list it saves them locally
        for a in alist:
    
            if dt.strptime(a.text, '%b %d, %Y') == row.date:
                
                try:
                    article = Article('https:' + a.attrs['href'])
                    article.download()
                    article.parse()
                except:
                    print("can't proces")

                try:
                    data.loc[index, 'title'] = article.title
                    if len(article.authors) > 0: 
                        data.loc[index, 'author'] = article.authors[0]
                    data.loc[index, 'published'] = article.publish_date
                    data.loc[index, 'text'] = article.text
                    data.loc[index, 'cleaned'] = 1
                except:
                    data.loc[index, 'cleaned'] = 0

    return data

In [ ]:
start = tm.time()
old_parse = old_parse(12706, chosen)
span = tm.time() - start
print('completed in ', span, ' seconds')

In [ ]:
old_parse.to_csv('test_parsing.csv')